# 资金费率套利策略

## 策略详情
---
### 原则
- 现货做多, 期货做空.
- 需要购买的交易对数量小于符合要求的交易对数量



### 逻辑步骤
- 设置阈值:
  - 可买入的资金费率最小值. `min_2_buy`  (建仓时使用: 对任何大于这个值的币种, 我们都可以买入)
  - 可卖出的资金费率最大值. `max_2_sell` (平仓时使用: 对任何小于这个值的持仓币, 我们都可以卖出)
- 创建两个消费者
  - 买入消费者
    - 当`symbol_holding_now < symbol_holding_max`时执行下面的行为
    - 遍历所有`usdt本位永续合约`, 获取所有资金费率大于`min_2_buy`的币种.
    - 排除已持有的交易对. 假设剩余数量为`symbol_count`
    - 查询交易对14天内资金费率历史
    - 按照14天内资金费率为正的比例从小到大排序交易对, 得到列表`list14`
    - 从`list14`末尾中取`len(list14) * pick_ratio`数量的子集`sublist14`
    - 对`sublist14`按照3天内资金费率为正的比例从小到大排序, 得到列表`list3`
    - 从`list3`末尾中取`symbol_holding_max - symbol_holding_now`数量的子集`symbols`
    - 购买`symbols`中的所有交易对 - 期货买空, 现货做多
  - 卖出消费者
    - 现货和期货价差
- `pick_ratio`计算法
  - 符合资金费率要求的交易对数量: `symbol_count`
  - 需要购买的交易对种类数量: `count = symbol_holding_max - symbol_holding_now`
  - 要求:
    - (count / symbol_count) < pick_ratio <= 1
  - 这是一个反比例函数, **只有当`(count / symbol_count) <= 1`时才有解**.



---
#### 变量说明
- `min_2_buy` - 可以买入的资金费率最小值
- `max_2_sell` - 可以卖出的资金费率最大值
- `symbols_holding_max` - 需要持有的交易对总数
- `symbols_holding_now` - 已持有的交易对数量
- `symbols_holding` - 已持有的交易对集合
- `symbols_count` - 资金费率大于`min_2_buy`且当前没有持有的交易对数量
- `list14` - 已排序的交易对集合, 排序方式: 14日内资金费率为正的比例由小到大
- `pick_ratio` - 筛选比例, 从`list14`中筛选交易对的比例
- `sublist14` - 从`list14`中选取的交易对
- `list3` - 已排序的交易对集合, 排序方式: 3日内资金费率为正的比例由小到大
- `count` - 需要购买的交易对数量
- `symbols` - 需要购买的交易对集合

In [1]:
import json
import os
import random
import requests
import urllib
import hmac
import hashlib
import csv
from enum import Enum
from datetime import datetime, timedelta
from decimal import Decimal, ROUND_CEILING

In [2]:
# global configuration
config = {
    'proxies': {
        'http': "socks5://127.0.0.1:9511", 
        'https': "socks5://127.0.0.1:9511"
    },
    # 是否使用代理
    'use_proxy': True,
    # 更新交易所支持的交易对集合的时间间隔, 单位: 秒
    'interval_2_update_fexchange': 60 * 10,
    # 上次更新交易对集合的时间
    'last_time_2_update_fexchange': None,
    # 期货交易的规则信息
    'fexchange_info': [],
    # 币安期货api的base_url
    'fbase_url': 'https://fapi.binance.com',
    # 币安现货api的base_url
    'base_url': 'https://api.binance.com',
    # 可以买入的资金费率最小值
    'min_2_buy': 0.0001,
    # 当前持有的交易对
    'symbols_holding': [],
    # 可持有的交易对最大数量
    'symbols_holding_max': 6,
    # binance api_key
    'binance_api_key': os.getenv('B_API_KEY'),
    # binance secrete_key
    'binance_secrete_key': os.getenv('B_SECRET_KEY'),
    # 交易记录文件保存位置
    'trading_record_path': 'trading.records/binance.csv',
    # database 根目录
    'database_path': 'databases'
}


class TransferType(Enum):
    """
    划转方向
    """
    MAIN_UMFUTURE = 'MAIN_UMFUTURE'  # 现货账户->期货账户
    MAIN_FUNDING = 'MAIN_FUNDING'  # 现货账户->资金账户

    UMFUTURE_MAIN = 'UMFUTURE_MAIN'  # 期货账户->现货账户
    UMFUTURE_FUNDING = 'UMFUTURE_FUNDING' # 期货账户->资金账户

    FUNDING_MAIN = 'FUNDING_MAIN'  # 资金账户->现货账户
    FUNDING_UMFUTURE = 'FUNDING_UMFUTURE'  # 资金账户->期货账户


class TradeSide(Enum):
    """
    交易方向
    """
    BUY = 'BUY'
    SELL = 'SELL'


class OrderType(Enum):
    """
    交易的订单类型
    """
    LIMIT = 'LIMIT'  # 限价单: 价格符合或低于限价则执行限价买单; 价格符合或高于限价则执行限价卖单.
    MARKET = 'MARKET'  # 市价单: 按照当前市场价格交易
    # mark: 这里还有另外5种类型的订单类型


class TimeInForce(Enum):
    """
    订单失效的方式
    """
    GOOD_TILL_CANCEL = 'GTC'  # 直到订单成交为止. 订单会一直有效, 直到被成交或者取消.
    IMMEDIATE_OR_CANCEL = 'IOC'  # 无法立即成交的部分将会被撤销. 订单失效前会尽量多的成交.
    FILL_OR_KILL = 'FOK'  # 无法全部立即成交就撤销. 如果无法全部成交, 订单就会失效.


class PositionSide(Enum):
    """
    期货持仓方向
    """
    BOTH = 'BOTH'  # 单一持仓方向
    LONG = 'LONG'  # 做多
    SHORT = 'SHORT'  # 做空

In [3]:
# the function of getting pick_ratio
def cal_pick_ratio(symbol_count, count):
    """
    @param symbol_count 资金费率大于`min_2_buy`且当前没有持有的交易对数量
    @param count 需要购买的交易对数量
    @return pick_ratio, (count / symbol_count) < pick_ratio <= 1
    """
    return 1 if count > symbol_count else random.uniform(count / symbol_count, 1)


# 用来保存逻辑处理过程中重要的值
value_stack = {}
def keep_value(name, value):
    """
    保存代码逻辑处理过程中生成的重要的值, 方便调试查看
    """
    value_stack[name] = value
    return


class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return str(obj)
        return json.JSONEncoder.default(self, obj)


def save_data():
    """
    保存内存数据到数据库
    """
    database_path = os.path.join(os.getcwd(), config['database_path'])
    if not os.path.exists(database_path):
        os.makedirs(database_path)
    json.dump(config, open(os.path.join(database_path, 'config.json'), 'a+'))


def restore_data():
    database_path = os.path.join(os.getcwd(), config['database_path'])
    if not os.path.exists(database_path):
        return
    old_config = json.load(open(os.path.join(database_path, 'config.json'), 'r'))
    config['symbols_holding'] = old_config['symbols_holding']

# todo 初始化时加载

In [4]:
def api_get(url: str, params: {} = None, need_key: bool = False, need_sign: bool = False):
    """
    requesting api by GET method
    
    :param url: the url to get
    :param params: query params
    :param need_key: 是否需要api_key, 默认不需要
    :param need_sign: 是否需要签名参数, 默认不需要
    :return: a object of class Response
    """
    proxies = config['proxies'] if config['use_proxy'] else {}
    headers = { 'X-MBX-APIKEY': config['binance_api_key'] } if need_key else {}
    if need_sign:
        params = sign(params)
    print('api_get', 'url:', url, 'params:', params)
    return requests.get(url, params=params, proxies=proxies, headers=headers)


def api_post(url: str, params: {} = None, need_key: bool = False, need_sign: bool = False):
    """
    requesting api by POST method
    
    :param url: the url to get
    :param params: query params
    :param need_key: 是否需要api_key, 默认不需要
    :param need_sign: 是否需要签名参数, 默认不需要
    :return: a object of class Response
    """
    proxies = config['proxies'] if config['use_proxy'] else {}
    headers = { 'X-MBX-APIKEY': config['binance_api_key'] } if need_key else {}
    if need_sign:
        params = sign(params)
    print('api_post', 'url:', url, 'params:', params)
    return requests.post(url, data=params, proxies=proxies, headers=headers)


def sign(params: {} = None):
    """
    为参数生成签名
    
    :param params: 网络请求参数
    """
    if not params:
        params = {}
    
    params['timestamp'] = int((datetime.now() + timedelta(seconds=1)).timestamp() * 1000)
    params['signature'] = hmac.new(
        key=config['binance_secrete_key'].encode('utf-8'),
        msg=urllib.parse.urlencode(query=params).encode('utf-8'),
        digestmod=hashlib.sha256
    ).hexdigest()
    return params

In [23]:
fbase_url = config['fbase_url']
base_url = config['base_url']


def get_fexchange_info():
    """
    更新期货交易规则和交易对信息
    """
    # too fast to update it again
    current = datetime.now().timestamp()
    last_updating_time = config['last_time_2_update_fexchange']
    if last_updating_time and current - last_updating_time < config['interval_2_update_fexchange'] and len(config['fexchange_info']) > 0:
        return config['fexchange_info']

    # todo catch http error
    exchange_info_url = '/fapi/v1/exchangeInfo'
    response = api_get(fbase_url + exchange_info_url)

    fexchange_info = json.loads(response.text)
    fsymbols_info = {}
    for symbol_item in fexchange_info['symbols']:
        symbol_item['filters'] = { item['filterType']: item for item in symbol_item['filters'] }
        fsymbols_info[symbol_item['symbol']] = symbol_item
    fexchange_info['symbols'] = fsymbols_info

    config['fexchange_info'] = fexchange_info
    config['last_time_2_update_fexchange'] = current
    return fexchange_info


def get_exchange_info(symbols: []):
    """
    获取现货时长的交易规范
    """
    if not symbols:
        raise ValueError('ERROR: 交易对信息不能为空!')
    
    exchange_info_url = '/api/v3/exchangeInfo'
    params = {
        'symbols': '['+','.join(f'"{symbol}"' for symbol in symbols)+']'
    }
    response = api_get(base_url + exchange_info_url, params=params)

    exchange_info = json.loads(response.text)
    symbols_info = {}
    for symbol_item in exchange_info['symbols']:
        symbol_item['filters'] = { item['filterType']: item for item in symbol_item['filters'] }
        symbols_info[symbol_item['symbol']] = symbol_item
    exchange_info['symbols'] = symbols_info
    return exchange_info


def get_premium_contracts(symbol: str = None):
    """
    get premium contract by the symbol
    
    :param symbol: 需要查询的交易对. 可空, 如果为空则查询所有交易对.
    """
    
    premium_index = '/fapi/v1/premiumIndex'
    response = api_get(fbase_url + premium_index, params={'symbol': symbol})
    # todo catch http error
    premium_contracts = json.loads(response.text)
    
    # 只选择USDT类的合约
    if not symbol:
        premium_contracts = [premium_contract for premium_contract in premium_contracts if premium_contract['symbol'].endswith('USDT')]
    return premium_contracts


def get_funding_rate_history(symbol: str):
    """
    get funding rate history of 14 days by the symbol
    
    :param symbol: 需要查询的交易对.
    """
    now = datetime.now()
    day_interval = timedelta(days=14)
    params = {
        'symbol': symbol,
        'startTime': int((now - day_interval).timestamp() * 1000),
        'endTime': int(now.timestamp() * 1000),
    }
    
    funding_rate = '/fapi/v1/fundingRate'
    response = api_get(fbase_url + funding_rate, params=params)
    funding_rate_history = json.loads(response.text)
    return funding_rate_history


def wallet_transfer(transfer_type: TransferType, asset: str, amount: float):
    """
    在币安钱包之间的进行金额的互转
    
    :param transfer_type: 划转类型
    :param asset: 划转资产名称
    :param amount: 划转的金额
    """
    params = {
        'type': transfer_type.value,
        'asset': asset,
        'amount': amount
    }
    
    transfer_url = '/sapi/v1/asset/transfer'
    response = api_post(base_url + transfer_url, params=params, need_key=True, need_sign=True)
    transfer_result = json.loads(response.text)
    if 'tranId' not in transfer_result:
        # todo 需要怎么处理划转失败的结果, 现在还没有想到合适的方式, 放着
        print('ERROR:', 'transfer error -', response.text)
    return transfer_result


def account_detail():
    """
    获取现货账户信息
    """
    account_url = '/api/v3/account'
    response = api_get(base_url + account_url, need_key=True, need_sign=True)
    account = json.loads(response.text)
    account['balances'] = [balance for balance in account['balances'] 
                           if float(balance['free']) > 0 
                           or float(balance['locked']) > 0
                          ]
    return account


def faccount_detail():
    """
    获取期货账户信息
    """
    faccount_url = '/fapi/v2/account'
    response = api_get(fbase_url + faccount_url, need_key=True, need_sign=True)
    faccount = json.loads(response.text)
    # todo 筛选我具有的资产的方式可能不对
    faccount['assets'] = [asset for asset in faccount['assets'] 
                          if float(asset['availableBalance']) > 0
                          or float(asset['crossWalletBalance']) > 0
                          or float(asset['maxWithdrawAmount']) > 0
                          or float(asset['walletBalance']) > 0
                         ]
    # todo positions
    return faccount


def get_ticker_depth(symbol: str, limit: int = 50, is_future: bool = False):
    """
    获取交易对的订单深度信息
    
    :param symbol: 交易对名称
    :param limit: 最大深度限制
    :param is_future: 是否是查询期货交易订单
    """
    c_ticker_url = '/api/v3/depth' if not is_future else '/fapi/v1/depth'
    c_base_url = base_url if not is_future else fbase_url
    response = api_get(c_base_url + c_ticker_url, params={'symbol': symbol, 'limit': limit})
    return json.loads(response.text)


def trading(symbol: str, trade_side: TradeSide, order_type: OrderType, time_in_force: TimeInForce, quantity: Decimal, price: Decimal):
    """
    现货交易方法

    :param symbol: 交易对名称.
    :param trade_side: 交易方式. 包括: 买入, 卖出.
    :param order_type: 订单类型. 包括: 限价单, 市价单等.
    :param time_in_force: 订单失效方式. 包括: 长期有效, 部分失效, 全部失效.
    :param quantity: 购买交易对的数量.
    :param price: 购买交易对的单价.
    :return: FULL格式的成交结果.
    """

    trading_url = '/api/v3/order'
    params = {
        'symbol': symbol,
        'side': trade_side.value,
        'type': order_type.value,
        'timeInForce': time_in_force.value,
        'quantity': quantity,
        'price': price
    }

    response = api_post(base_url + trading_url, params=params, need_key=True, need_sign=True)
    return json.loads(response.text)


def ftrading(symbol: str, trade_side: TradeSide, position_side: PositionSide, order_type: OrderType, time_in_force: TimeInForce, quantity: Decimal, price: Decimal):
    """
    期货交易方法

    :param symbol: 交易对名称.
    :param trade_side: 交易方式. 包括: 买入, 卖出.
    :param position_side: 持仓方向. 包括: 单项持仓, 做多, 做空.
    :param order_type: 订单类型. 包括: 限价单, 市价单等.
    :param time_in_force: 订单失效方式. 包括: 长期有效, 部分失效, 全部失效.
    :param quantity: 购买交易对的数量.
    :param price: 购买交易对的单价.
    :return: RESULT格式的成交结果.
    """
    ftrading_url = '/fapi/v1/order'
    params = {
        'symbol': symbol,
        'side': trade_side.value,
        'positionSide': position_side.value,
        'type': order_type.value,
        'timeInForce': time_in_force.value,
        'quantity': quantity,
        'price': price,
        'newOrderRespType': 'RESULT'
    }

    response = api_post(fbase_url + ftrading_url, params=params, need_key=True, need_sign=True)
    return json.loads(response.text)

In [59]:
"""
- 买家逻辑
    - 当`symbol_holding_now < symbol_holding_max`时执行下面的行为
    - 遍历所有`usdt本位永续合约`, 获取所有资金费率大于`min_2_buy`的币种.
    - 排除已持有的交易对. 假设剩余数量为`symbol_count`
    - 查询交易对14天内资金费率历史
    - 按照14天内资金费率为正的比例从小到大排序交易对, 得到列表`list14`
    - 从`list14`末尾中取`len(list14) * pick_ratio`数量的子集`sublist14`
    - 对`sublist14`按照3天内资金费率为正的比例从小到大排序, 得到列表`list3`
    - 从`list3`末尾中取`symbol_holding_max - symbol_holding_now`数量的子集`symbols`
    - 购买`symbols`中的所有交易对 - 期货买空, 现货做多
"""

# the buyer who wants to buy new symbols
def buyer():
    """
    永续合约购买者
    """
    symbols_holding = config['symbols_holding']
    symbols_holding_max = config['symbols_holding_max']
    # held enough symbols
    if len(symbols_holding) >= symbols_holding_max:
        print('buyer exits. holding enough symbols already!')
        return
    
    # 获取交易所支持的所有永续交易对
    premium_contracts = get_premium_contracts()
    keep_value('premium_contracts', premium_contracts)
    # 大于最小买入值 and 排除已购买的交易对
    symbols_watching = [premium_contract['symbol'] for premium_contract in premium_contracts if float(premium_contract['lastFundingRate']) >= config['min_2_buy']]
    symbols_watching = list(set(symbols_watching).difference(set(symbols_holding)))
    keep_value('symbols_watching', symbols_watching)
    symbols_to_buy = find_most_wanted_symbols(symbols_watching, symbols_holding_max, len(symbols_holding))
    print('buyer wants to purchase:', symbols_to_buy)
    keep_value('symbols_to_buy', symbols_to_buy)
    purchase_symbols(symbols_to_buy)
    return


def find_most_wanted_symbols(symbols_watching: [], holding_max: int, holding_count: int):
    """
    找到最合适购买的交易对
    
    todo:
    1. 没有考虑期货和现货的价差
    
    :param symbols_watching: 有意向购买的交易对信息
    :param holding_max: 可持有的交易对最大数量
    :param holding_count: 已持有交易对的数量
    """
    # todo 这个方法返回太慢了, 需要优化. tips: 8小时才会更新一次的
    funding_rate_histories = {symbol: get_funding_rate_history(symbol) for symbol in symbols_watching}
    
    import time
    three_days_before = time.mktime((datetime.today() - timedelta(days=3)).date().timetuple()) * 1000
    funding_rate_statistics = {}
    for symbol, funding_rates in funding_rate_histories.items():
        positive_in_14 = 0
        positive_in_3 = 0
        
        for funding_rate in funding_rates:
            if float(funding_rate['fundingRate']) > 0:
                positive_in_14 += 1
                if funding_rate['fundingTime'] > three_days_before:
                    positive_in_3 += 1
        
        # todo 如果总记录数太少, 正值率为100%, 数据能用吗?
        funding_rate_statistics[symbol] = {
            'positive_in_14': positive_in_14,
            'positive_in_3': positive_in_3,
            'funding_rates_count': len(funding_rates)
        }
    
    # 按照14天内资金费率为正的比例从小到大排序
    sorted_funding_rates_by_14 = sorted(funding_rate_statistics.items(), key=lambda item: item[1]['positive_in_14'] / item[1]['funding_rates_count'])
    keep_value('sorted_funding_rates_by_14', sorted_funding_rates_by_14)
    buying_symbols_len_max = holding_max - holding_count
    pick_ratio = cal_pick_ratio(len(symbols_watching), buying_symbols_len_max)
    picked_funding_rates = sorted_funding_rates_by_14[len(sorted_funding_rates_by_14) - round(len(sorted_funding_rates_by_14) * pick_ratio):]
    keep_value('pick_ratio', pick_ratio)
    keep_value('picked_funding_rates', picked_funding_rates)
    
    # 按照3天内资金费率为正的比例从小到大排序
    sorted_funding_rates_by_3 = sorted(picked_funding_rates, key=lambda item: item[1]['positive_in_3'] / item[1]['funding_rates_count'])
    buying_symbols_len = buying_symbols_len_max if buying_symbols_len_max < len(sorted_funding_rates_by_3) else len(sorted_funding_rates_by_3)
    keep_value('buying_symbols_len', buying_symbols_len)
    keep_value('sorted_funding_rates_by_3', sorted_funding_rates_by_3)
    # todo 没有返回的内容里面也有一些很好的交易对, 应该加入对期货价和现货价差值的筛选
    return [item[0] for item in sorted_funding_rates_by_3[len(sorted_funding_rates_by_3) - buying_symbols_len:]]


def bargain(symbols_to_buy: []):
    """
    获取购买交易对的价格和数量

    :param symbols_to_buy: 交易对集合
    :return: 每个交易对最优价格和数量的集合
    """
    # 获取交易规则信息
    fexchange_info = get_fexchange_info()
    exchange_info = get_exchange_info(symbols_to_buy)

    shopping_list = []
    for target_symbol in symbols_to_buy:
        price = Decimal(get_ticker_depth(target_symbol, 10)['asks'][1][0])
        symbol_info = exchange_info['symbols'][target_symbol]
        filters = symbol_info['filters']
        fprice = Decimal(get_ticker_depth(target_symbol, 100, True)['asks'][-1][0])
        fsymbol_info = fexchange_info['symbols'][target_symbol]
        ffilters = fsymbol_info['filters']
        lot_min_quantity = max(Decimal(ffilters['LOT_SIZE']['minQty']), Decimal(filters['LOT_SIZE']['minQty']))  # 满足订单尺寸的最小数量
        notional_min_quantity = max(
            Decimal(ffilters['MIN_NOTIONAL']['notional']) / fprice,
            Decimal(filters['MIN_NOTIONAL']['minNotional']) / price)  # 满足名义价值的最小数量
        min_quantity = max(lot_min_quantity, notional_min_quantity).quantize(Decimal('0.'+'0'*fsymbol_info['quantityPrecision']), ROUND_CEILING)
        shopping_list.append({
            "symbol": target_symbol,
            'quantity': min_quantity,
            'fprice': fprice,
            'price': price,
            'costs': (fprice + price) * min_quantity
        })
    return sorted(shopping_list, key=lambda item: item['costs'])


def purchase_symbols(symbols_to_buy: []):
    """
    购买指定的交易对.
    
    1. 查询购买币的最小个数
    2. 获得当前订单深度
    3. 购买买入深度第一个值的价格 * 最小个数的币
    
    :param symbols_to_buy: 需要购买的交易对列表
    """
    # todo 暂时使用这个交易对进行购买
    shopping_list = bargain(symbols_to_buy)
    keep_value('shopping_list', shopping_list)

    for index in range(0, 1):
        shopping_item = shopping_list[index]
        # todo 购买应该发生在同一个事务中
        ftrading_result = ftrading(shopping_item['symbol'], TradeSide.SELL, PositionSide.SHORT, OrderType.LIMIT, TimeInForce.GOOD_TILL_CANCEL, shopping_item['quantity'], shopping_item['fprice'])
        trading_result = value_stack['trading_result']
        # trading(shopping_item['symbol'], TradeSide.BUY, OrderType.LIMIT, TimeInForce.GOOD_TILL_CANCEL, shopping_item['quantity'], shopping_item['price'])
        keep_value('ftrading_result', ftrading_result)
        keep_value('trading_result', trading_result)
        print('a buyer transaction: symbol name {}, quantity {}, spot price {}, future price {}, total costs {}'.format(
            shopping_item['symbol'], shopping_item['quantity'], shopping_item['price'], shopping_item['fprice'], shopping_item['costs']
        ))
        config['symbols_holding'].append(shopping_item['symbol'])
        keep_buying_record(ftrading_result, trading_result, shopping_item)


def keep_buying_record(ftrading_result, trading_result, shopping_item):
    """
    保存购买记录

    :param ftrading_result: 期货购买记录
    :param trading_result: 现货购买记录
    :param shopping_item: 交易对在购物清单上的信息
    """
    file_path = os.path.join(os.getcwd(), config['trading_record_path'])
    does_exists = os.path.exists(file_path)

    if not does_exists:
        dir_path = os.path.dirname(file_path)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    with open(file_path, 'a+') as fd:
        writer = csv.writer(fd)
        contents = []
        headers = 'sequence,symbol,exchange_type,trading_side,position_side,target_quantity,target_price,target_costs,order_id,quantity,price,costs,trading_fees,trading_result'.split(',')
        if not does_exists:
            contents.append(headers)
        sequence = int(datetime.now().timestamp() * 1000000)
        if ftrading_result:
            try:
                contents.append([
                    sequence,
                    shopping_item['symbol'],
                    'FUTURES',
                    ftrading_result['side'],
                    ftrading_result['positionSide'],
                    shopping_item['quantity'],
                    shopping_item['fprice'],
                    shopping_item['fprice'] * shopping_item['quantity'],
                    ftrading_result['orderId'],
                    ftrading_result['executedQty'],
                    ftrading_result['avgPrice'],
                    ftrading_result['cumQuote'],
                    'trading_fees',
                    json.dumps(ftrading_result)
                ])
            except Exception as e:
                print(e)
                error_content = [None] * len(headers)
                error_content[0] = sequence
                error_content[1] = shopping_item['symbol']
                error_content[2] = 'FUTURES'
                error_content[-1] = json.dumps(ftrading_result)
                contents.append(error_content)

        if trading_result:
            try:
                contents.append([
                    sequence,
                    shopping_item['symbol'],
                    'SPOT',
                    trading_result['side'],
                    'LONG',
                    shopping_item['quantity'],
                    shopping_item['price'],
                    shopping_item['price'] * shopping_item['quantity'],
                    trading_result['orderId'],
                    trading_result['executedQty'],
                    trading_result['price'],
                    trading_result['cummulativeQuoteQty'],
                    'trading_fees' if not trading_result['fills'] else sum([Decimal(item['commission']) for item in trading_result['fills']]),
                    json.dumps(trading_result)
                ])
            except Exception as e:
                print(e)
                error_content = [None] * len(headers)
                error_content[0] = sequence
                error_content[1] = shopping_item['symbol']
                error_content[2] = 'SPOT'
                error_content[-1] = json.dumps(trading_result)
                contents.append(error_content)
        writer.writerows(contents)

In [61]:
# print(json.dumps(config['fexchange_info'], sort_keys=True, indent=4, separators=(',', ':')))
# print(json.dumps(bargain(value_stack['symbols_to_buy']), sort_keys=True, indent=4, separators=(',', ':')))
# purchase_symbols(value_stack['symbols_to_buy'])
# buyer()
del config['symbols_holding'][0]

In [ ]:
# the seller who sells holding symbols

## 补充知识

### 资金费率
资金费率是数字货币期货市场独有的概念. 数字货币的期货被称为永续合约.

资金费率的影响因素包括:
- 综合利率
  - 这个是固定不变的.
- 永续合约和现货之间的溢价. (溢价 = 永续合约价 - 现货价)
  - 溢价越大, 资金费率越高, 支付的金额越多.
  
资金费率为正:
- 永续合约价格 > 现货价格
- `多头`支付金额给`空头`
资金费率为负:
- 永续合约价格 < 现货价格
- `空头`支付金额给`多头`

### 资金费率历史
- [币安](https://www.binance.com/cn/futures/funding-history/1)

### 结算公式
结算资金 = 持仓名义价值 * 资金费率
持仓名义价值 = 标记价格 * 持有合约数量

## Todo
- 需要考虑因为杠杆导致的被系统平仓的风险
  - 什么是杠杆率
  - 杠杆下波动对资金的影响
  - 系统平仓逻辑
  - 实时监控, 防止被自动平仓
  - **所有的文章里都提到了小心杠杆**  
- 需要考虑开仓/平仓的**手续费**, 另外为了成功交易还需要设置**滑点**
- 逻辑步骤 - 买入消费者 - 购买`symbols`中的所有交易对
  - 购买涉及到杠杆的知识, 需要在实现时学习一下
  - 如何设计购买策略, 才能把所有的交易对**全部都买到**呢
  - 币安交易的手续费怎么算, 没看懂. [地址](https://www.binance.com/zh-CN/support/faq/360033544231)
- 是否要对14, 3天内资金费率为正的比例设置最小值呢
- 落库数据
  - 购买的交易历史落库, 重新启动时加载
    - 方便统计收益率之类的数据
  - 持有的交易对
- 报错导致数据错误的位置添加try except
- 恢复错误数据
- 合约购买出错的原因



## V2
- `p14`和`p3`计算法
  - 符合资金费率要求的交易对数量: `symbol_count`
  - 需要购买的交易对种类数量: `count = symbol_holding_max - symbol_holding_now`
  - 要求:
    - 0 < p14, p3 <= 1
    - p14 * p3 >= count / symbol_count
  - 这是一个反比例函数, **只有当`(count / symbol_count) <= 1`时才有解**.